# Geocoding API


In [68]:
import herepy
import pandas as pd
from tqdm import tqdm 
import time
import math

## Tuguegarao

In [165]:
df = pd.read_csv('Mago_copy_CaVolun_Initiative_105720201115.csv')

In [166]:
df = df[(df['USABLE ADDRESS'] == 'Yes') | (df['REVISED ADDRESS'].notnull())]

In [167]:
df = df[df['RESCUE STATUS'] != 'Rescued']
df = df[df['RESCUE STATUS'] != 'Ongoing']
df = df[df['RESCUE STATUS'] != 'No need']
df = df[df['RESCUE STATUS'] != 'Safe at home']

In [168]:
df['RESCUE STATUS'].unique()

array(['Not yet responded', nan, 'Not Yet Responded',
       'Second floor na baha', '2 adults 1 kid, nasa bubong',
       'Families - sobrang gutom', 'Cannot contact',
       'cannot be rescued due to location', '2:08',
       'trapped on the 2nd floor', 'not yet rescued',
       'landmark: pinakadulo malapit sa evas',
       'URGENT! may inatake sa puso',
       'flashlight standby; di na kayang sumigaw',
       'no other info; not yet rescued since 8pm',
       'light can be seen at jl fuel', 'No info', 'Wala nang battery',
       'need rescue asap', 'NEEDS URGENT HELP', 'Timoy Agub'],
      dtype=object)

In [169]:
df = df[df['address'] != '']

In [189]:
df['AREA'] = df['AREA'].replace('Cetro', 'Centro')
df['AREA'] = df['AREA'].replace('North: Carig-Caritan', 'Carig-Caritan')
df['AREA'] = df['AREA'].replace('West: Buntun-Ugac', 'Buntun-Ugac')

In [215]:
df['AREA'].unique()

array(['East: Namabbalan-Capatan', 'Carig-Caritan', 'Buntun-Ugac',
       'Centro', nan, 'Centro: Centro 1-12', 'Pengue', 'Linao East',
       'Linao Norte, Tuguegarao', 'Linao, Tuguegarao', 'Linao Norte',
       'Linao Norte ', 'Linao West', 'Annafuan East', 'Enrile Cagayan',
       'linao norte', 'Buntun', 'LINAO NORTH', 'Balzain', 'Linao east',
       'Annafunan West', 'San Gabriel', 'Liano EAST tapat',
       'Balzain East Tuguegarao City', 'Cataggaman Nuevo', 'Cattaggamman',
       'Tuguegarao City', 'Annafunan East', 'Linao east ', 'Balzain East',
       'linao east ', 'pinakadulo ', 'Balzain west', 'Tan', 'Linao North',
       'Tanza', 'Ugac Sur', 'balzain east ', 'West Area', 'Liano East ',
       'Pallua Sur, near elem school papasok sa kanto ',
       'Brgy. San Gabriel'], dtype=object)

In [192]:
df = df[df['address'] != '']

## Google Maps Geocoding

In [70]:
import googlemaps
from datetime import datetime

# PUT YOUR OWN API KEY
gmaps = googlemaps.Client(key='')

In [178]:
#Testing
geocode_result = gmaps.geocode('zone 4 duhat street barangay capatan ,Namabbalan-Capatan, Tuguegarao City, Cagayan')
geocode_result[0]

In [195]:
lat_list = []
lng_list = []
with tqdm(total=df.shape[0]) as pbar:
    for index, row in df.iterrows():
        pbar.update(1)
                
        try:
            if pd.isnull(row['REVISED ADDRESS']):
                address = row['address']
            else:
                address = row['REVISED ADDRESS']
        except:
            address = row['REVISED ADDRESS']
            
        if pd.isnull(row['AREA']) == False:
            address = address + ', ' + row['AREA']
            
        if "tuguegarao" in address.lower():
            address = address + ', Cagayan'
        else:
            address = address + ', Tuguegarao City, Cagayan'
            
#         print(address)
                    
        geocode_result = gmaps.geocode(address)
        try:
            lat = geocode_result[0]['geometry']['location']['lat']
            long = geocode_result[0]['geometry']['location']['lng']
            
            lat_list.append(lat)
            lng_list.append(long)
        except:
            lat_list.append(0)
            lng_list.append(0)
        time.sleep(2)

100%|██████████| 336/336 [13:36<00:00,  2.43s/it]


In [196]:
df['lat'] = lat_list
df['lng'] = lng_list

In [197]:
df.to_csv('tuguegeocoded_locations_v2.csv')

In [30]:
df.to_csv('tuguegeocoded_locations_v2.csv')

## GEOPANDAS

In [3]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd

#### Converting our scraped data to geopandas format

In [131]:
df = pd.read_csv('tuguegeocoded_locations_v2.csv')

In [132]:
df2 = pd.read_csv('cagayan_other_locations_v1.csv')

In [133]:
df = df[['address','AREA','NOTES',"Contact Number\n  ANOTHER NOTE: Countless rescuers na ang namatay dahil naelectrocute ng generators. INFORM PEOPLE SA CAGAYAN TO TURN OFF THEIR GENERATORS. Buhay muna ng rescuers bago ang kuryente!!", 'PERSONS TO RESCUE', 'DATE AND TIME CONTACTED','lat','lng']]

In [134]:
df2 = df2[['address','AREA','NOTES','CONTACT NUMBER','PERSONS TO RESCUE','DATE AND TIME CONTACTED', 'lat','lng']]

In [135]:
df.columns = df2.columns

In [136]:
df = df2.append(df)

In [137]:
# Converting the 
df['lat'] = df['lat'].astype(float)
df['lng'] = df['lng'].astype(float)

In [138]:
df = df[(df['lng'] >= 120.68) & (df['lng'] <= 122.4)]
df = df[(df['lat'] >= 17.5) & (df['lat'] <= 19.6)]

In [139]:
gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.lng, df.lat))

# Mapping

In [140]:
import folium
import folium.plugins as plugins


In [141]:
df = df.reset_index(drop = True)

In [142]:
df1 = df[~df.geometry.duplicated()]
df2 = df[df.geometry.duplicated()]

In [143]:
coo = []
info = []

for geo in df2['geometry'].unique():
    coo.append(geo)
    info.append(df[df['geometry'] == geo][['address', 'AREA','CONTACT NUMBER', 'PERSONS TO RESCUE','DATE AND TIME CONTACTED']].to_html())
    
tables = pd.DataFrame({'geometry': coo, 'info': info})

coo = []
info = []

for geo in df1['geometry'].unique():
    coo.append(geo)
    info.append(df[df['geometry'] == geo][['address', 'AREA','CONTACT NUMBER', 'PERSONS TO RESCUE','DATE AND TIME CONTACTED']].to_html())
    
tables = tables.append(pd.DataFrame({'geometry': coo, 'info': info}))

In [144]:
tables = tables.reset_index(drop = True)
tables = gpd.GeoDataFrame(tables, geometry='geometry')


In [145]:
gdf = gdf.merge(tables, on='geometry').drop_duplicates().reset_index(drop = 'True')

In [162]:
df['info'] = [' DETAILS: '.join(i) for i in zip(df['address'].astype(str),df['NOTES'].astype(str))]
df['info'] = [' AREA: '.join(i) for i in zip(df['info'].astype(str),df['AREA'].astype(str))]
df['info'] = [' CONTACT NUMBER: '.join(i) for i in zip(df['info'].astype(str),df["CONTACT NUMBER"].astype(str))]
df['info'] = [' PERSONS TO RESCUE: '.join(i) for i in zip(df['info'].astype(str),df["PERSONS TO RESCUE"].astype(str))]
df['info'] = [' DATE AND TIME CONTACTED: '.join(i) for i in zip(df['info'].astype(str),df["DATE AND TIME CONTACTED"].astype(str))]
df['info'] = 'ADDRESS: ' + df['info']

In [146]:
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
}

In [155]:
from IPython.display import IFrame


In [182]:
gdf['test'] = "<div style='height:100px;overflow:auto;'>"+ gdf['info'] +"</div>" 

In [183]:
gdf

,address,AREA,NOTES,CONTACT NUMBER,PERSONS TO RESCUE,DATE AND TIME CONTACTED,lat,lng,geometry,info,test
0,brgy. damurog zone 4,Alcala,NaN,NaN,NaN,NaN,17.878753,121.665383,POINT (121.66538 17.87875),"<table border=""1"" class=""dataframe"">\n <thead...",<div style='height:100px;overflow:auto;'><tabl...
1,Zone2,Alcala,di makontact kanina pa tanghali,9751500123,Jeloubelle Conceja,NaN,17.904972,121.651126,POINT (121.65113 17.90497),"<table border=""1"" class=""dataframe"">\n <thead...",<div style='height:100px;overflow:auto;'><tabl...
2,NaN,Alcala,NaN,NaN,Claudilyn,NaN,17.904972,121.651126,POINT (121.65113 17.90497),"<table border=""1"" class=""dataframe"">\n <thead...",<div style='height:100px;overflow:auto;'><tabl...
3,Baybayog,Alcala,with 2 yr old daughter,NaN,"Castro, Sheena",NaN,17.887315,121.705295,POINT (121.70530 17.88732),"<table border=""1"" class=""dataframe"">\n <thead...",<div style='height:100px;overflow:auto;'><tabl...
4,"Zone 2, Alcala","Alcala, Zone 2, Calantac",May 2 bata,9751500123,"Conceja, Jeloubelle",NaN,17.896843,121.725246,POINT (121.72525 17.89684),"<table border=""1"" class=""dataframe"">\n <thead...",<div style='height:100px;overflow:auto;'><tabl...
...,...,...,...,...,...,...,...,...,...,...,...
413,Liban St. BAlzain,NaN,nagriring lang,9956608829,"Canapi, Jeremy; Baggay Shell (buntis)",4:28 am 11/14/20,17.623427,121.733539,POINT (121.73354 17.62343),"<table border=""1"" class=""dataframe"">\n <thead...",<div style='height:100px;overflow:auto;'><tabl...
414,"victoria hills subv., atulayan norte, tuguegar...",NaN,NaN,"(2 kids, 3 teens, and one adult with dead bodies)",NaN,kyle corrales,17.660808,121.756372,POINT (121.75637 17.66081),"<table border=""1"" class=""dataframe"">\n <thead...",<div style='height:100px;overflow:auto;'><tabl...
415,"Victoria Hills Subdivision, Atulayan, Tuguegar...",NaN,NaN,Bitbit na daw po nya bangkay ng kapatid nya. P...,NaN,Kyle Zion Buenaventura Corales,17.660808,121.756372,POINT (121.75637 17.66081),"<table border=""1"" class=""dataframe"">\n <thead...",<div style='height:100px;overflow:auto;'><tabl...
416,"Core shelter Aguinaldo, cataggaman Pardo",NaN,NaN,"coordinator for all, may 25 families sa bagong...",9652348185,Sugarette Buenaobra Parayo,17.604844,121.699594,POINT (121.69959 17.60484),"<table border=""1"" class=""dataframe"">\n <thead...",<div style='height:100px;overflow:auto;'><tabl...


In [184]:
map1 = folium.Map(
    location=[17.6245371,121.7259032 ],
    zoom_start=10,
)


loc = 'Tuguegarao Rescue Request Mapping'
title_html = '''
             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(loc)   

map1.get_root().html.add_child(folium.Element(title_html))

loc = 'Last Updated: 7:11 PM 11-15-2020 <br> Mapped using Google Maps Geocoding.<br> Note that these are rough estimates of the coordinates <br> as identified by the Google API based from address description.<br> Data Source: <a href = "https://docs.google.com/spreadsheets/d/1WijcCj3nlbKPESdlcdotoP4H1hLyS3RwxJIkN_kUP3E/htmlview#"> CaVolun Initiative </a> <br> Generated by <a href = "https://github.com/magoanalytics/cagayan-rescue-mapping">Mago Analytics </a>'
title_html = '''
             <p align="center" style="font-size:9">{}</p>
             '''.format(loc)   

map1.get_root().html.add_child(folium.Element(title_html))

# Add custom basemaps
basemaps['Google Maps'].add_to(map1)

gdf.apply(lambda row:folium.CircleMarker(location=[row["lat"], row["lng"]], popup = row['test'],fill_color='red', height=100).add_to(map1), axis=1)
map1

In [185]:
map1.save('tuguegarao_v2.html')

## --------------------------------------------

## Other Districts

In [23]:
import os

In [63]:
df = pd.DataFrame()
for file in os.listdir('./Cagayan Districts Data'):
    if '.csv' in file:
        to_append = pd.read_csv('Cagayan Districts Data/' + file)[['LOCATION','AREA','REVISED ADDRESS','STATUS','RESCUE STATUS','PERSONS TO RESCUE', 'CONTACT NUMBER']]
        df = df.append(to_append)

In [76]:
lat_list = []
lng_list = []
with tqdm(total=df.shape[0]) as pbar:
    for index, row in df.iterrows():
        pbar.update(1)
                
        try:
            if pd.isnull(row['REVISED ADDRESS']):
                if pd.isnull(row['LOCATION']) == False:
                    address = row['LOCATION']
                else:
                    address = ''
            else:
                address = row['REVISED ADDRESS']
        except:
            address = row['AREA']
            
            
        if pd.isnull(row['AREA']) == False:
            address = address + ', ' + row['AREA']
            
        address = address + ', Cagayan'
            
#         print(address)
                    
        geocode_result = gmaps.geocode(address)
        try:
            lat = geocode_result[0]['geometry']['location']['lat']
            long = geocode_result[0]['geometry']['location']['lng']
            
            lat_list.append(lat)
            lng_list.append(long)
        except:
            lat_list.append(0)
            lng_list.append(0)
        time.sleep(2)

100%|██████████| 108/108 [04:21<00:00,  2.42s/it]


In [77]:
df['lat'] = lat_list
df['lng'] = lng_list

In [80]:
df.columns = (['address', 'AREA', 'REVISED ADDRESS','NOTES','RESCUE STATUS', 'PERSONS TO RESCUE', 'CONTACT NUMBER', 'lat','lng'])
df['DATE AND TIME CONTACTED'] = ''
df = df[df['RESCUE STATUS'].str.contains('Not')]

In [86]:
df.to_csv('cagayan_other_locations_v1.csv')